In [74]:
import os
import sys
import subprocess as subp
import json
import numpy as np
from scipy import stats, integrate
import scipy.io as sio
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor, as_completed
import nibabel as nib

In [12]:
# load good stuff
%matplotlib inline
%load_ext rpy2.ipython
%load_ext oct2py.ipython
sns.set(color_codes=True)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The oct2py.ipython extension is already loaded. To reload it, use:
  %reload_ext oct2py.ipython


In [13]:
# Add niak hcp and psomm to oactve path
%octave addpath(genpath('~/git/Misc'));
%octave build_path hcp niak psom

Adding library hcp to the search path.

Adding library niak to the search path.

Adding library psom to the search path.

In [14]:
#custom function for key sorting element
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]

In [28]:
# Set path 
#path_root = '/home/yassinebha/data/data_disk/Drive/HCP2/Solar_heritability/HCP_subtype/'
path_root = '/scratch/yassinebha/pleio/pleio_association/'
#  Set file pattern
pedig_file_ptrn = 'solar_{}_spm_pedigre.csv'
pheno_file_ptrn = 'solar_{}_spm_{}_pheno.csv'
pheno_clust = os.path.join(path_root,'hcp_bootstraped_pheno_nonan_norm_python_13.csv')
# Set task and trials
task_name = 'wm'
nb_sbt= 5
list_sbt = ['sub%s' % str(i+1) for i in range(nb_sbt)]
list_trial = ['2bk','0bk','contrast_2bk_vs_0bk']

In [16]:
list_sbt

['sub1', 'sub2', 'sub3', 'sub4', 'sub5']

In [34]:
# Load pedig file
pedig_df = pd.read_csv(os.path.join(path_root,pedig_file_ptrn.format(str.upper(task_name))))

In [37]:
# Load subtypes weights
sbt_weight_df = pd.read_csv(os.path.join(path_root,'solar_{}_spm_{}_pheno.csv'.format(str.upper(task_name),
                                                                                 str(nb_sbt))))
sbt_weight_df.count()

ID                             803
WM_2bk_sub1                    803
WM_2bk_sub2                    803
WM_2bk_sub3                    803
WM_2bk_sub4                    803
WM_2bk_sub5                    803
WM_contrast_2bk_vs_0bk_sub1    803
WM_contrast_2bk_vs_0bk_sub2    803
WM_contrast_2bk_vs_0bk_sub3    803
WM_contrast_2bk_vs_0bk_sub4    803
WM_contrast_2bk_vs_0bk_sub5    803
WM_0bk_sub1                    803
WM_0bk_sub2                    803
WM_0bk_sub3                    803
WM_0bk_sub4                    803
WM_0bk_sub5                    803
Age_in_Yrs                     803
Gender                         803
BMI                            803
WM_FD_mean                     803
WM_FD_scrubbed_mean            803
dtype: int64

In [61]:
# Load phenotype clustered file
pheno_clust_df = pd.read_csv(pheno_clust).rename(columns={'Subject': 'ID'})
pheno_clust_df.loc[:,'ID'] = 'HCP' + pheno_clust_df['ID'].astype(str)
pheno_clust_df.head()

,ID,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,Cluster_7,Cluster_8,Cluster_9,Cluster_10,Cluster_11,Cluster_12,Cluster_13
0,HCP100004,-1.051773,0.134739,-0.538317,1.185358,0.085162,-0.279920,0.466965,0.832836,-1.530547,-0.097661,0.398510,0.082329,-0.624942
1,HCP100206,-1.143145,-0.344595,1.301892,1.283574,1.436342,0.052662,0.706008,1.282970,-0.510366,0.205767,1.975280,-0.303875,0.064552
2,HCP100307,-0.528705,-0.639736,0.492726,0.522789,-0.713226,-0.952467,-0.559144,-0.166309,-0.140952,0.387172,-0.703027,-0.249928,-0.420702
3,HCP100408,-0.237051,1.173656,0.018709,0.092160,-0.034026,-0.521888,-0.109229,0.865643,0.221359,-0.082267,-0.589268,0.011946,0.191045
4,HCP100610,2.017489,0.891096,0.894110,-0.361182,-0.386394,-0.881930,-0.575076,0.206005,-0.197974,0.709730,0.203063,0.229882,1.150436


In [71]:
# Clusters list
list_pheno_clust = [x for x in pheno_clust_df.columns.get_values() if x!='ID']
list_pheno_clust

['Cluster_1',
 'Cluster_2',
 'Cluster_3',
 'Cluster_4',
 'Cluster_5',
 'Cluster_6',
 'Cluster_7',
 'Cluster_8',
 'Cluster_9',
 'Cluster_10',
 'Cluster_11',
 'Cluster_12',
 'Cluster_13']

In [46]:
# Merge pheno clusters with subtypes
sbt_pheno_df = pd.merge(sbt_weight_df,pheno_clust_df,how='left',left_on='ID',right_on='Subject')
sbt_pheno_df

ID                             803
WM_2bk_sub1                    803
WM_2bk_sub2                    803
WM_2bk_sub3                    803
WM_2bk_sub4                    803
WM_2bk_sub5                    803
WM_contrast_2bk_vs_0bk_sub1    803
WM_contrast_2bk_vs_0bk_sub2    803
WM_contrast_2bk_vs_0bk_sub3    803
WM_contrast_2bk_vs_0bk_sub4    803
WM_contrast_2bk_vs_0bk_sub5    803
WM_0bk_sub1                    803
WM_0bk_sub2                    803
WM_0bk_sub3                    803
WM_0bk_sub4                    803
WM_0bk_sub5                    803
Age_in_Yrs                     803
Gender                         803
BMI                            803
WM_FD_mean                     803
WM_FD_scrubbed_mean            803
Subject                        803
Cluster_1                      803
Cluster_2                      803
Cluster_3                      803
Cluster_4                      803
Cluster_5                      803
Cluster_6                      803
Cluster_7           

In [45]:
# check if any NaN value present
sbt_pheno_df.isnull().values.sum()

0

### Pleiothropy estimate:

In [72]:
import collections
import multiprocessing 
import datetime

In [81]:
#initiate empty dictionary
Pleio_Asso = collections.namedtuple('Pleio_Asso', [
    'out_dir',
    'pedig_f',
    'pheno_f',
    'var_1',
    'var_2',
    'covar_1'
])
pleio_asso = []

# pleio root folder
path_pleio = os.path.join(path_root,'pleio_{}_{}'.format(str(datetime.date.today()),
                                                        task_name))
if not os.path.exists(path_pleio):
    os.makedirs(path_pleio)

# create ouput folders and populate RhoG dictionnary
for ix_trial, trial in enumerate(list_trial):
    for ix_clust, clust in enumerate(list_pheno_clust):
        for ix_subt, subt in enumerate(list_sbt):
            print(trial,clust,subt)
            pheno_1 = clust
            pheno_2 = '{}_{}_{}'.format(str.upper(task_name),trial,subt)
            cov_1 = '{}_FD_scrubbed_mean'.format(str.upper(task_name))
        
            # output result folder 
            path_pleio_contrast = os.path.join(path_pleio,'{}_{}'.format(trial,subt))
            if not os.path.exists(path_pleio_contrast):
                os.makedirs(path_pleio_contrast)
            
            # copy needed files to output folder
            if not os.path.isfile(os.path.join(path_pleio_contrast,'se_univ_polygen.tcl')):
                subp.run(['cp',os.path.join(path_root,'se_univ_polygen.tcl'),path_pleio_contrast])
            if not os.path.isfile(os.path.join(path_pleio_contrast,'pleio_run.sh')):
                subp.run(['cp',os.path.join(path_root,'pleio_run.sh'),path_pleio_contrast])
        
        # pedegree
        if not os.path.isfile(os.path.join(path_pleio_contrast,pedig_file_ptrn.format(str.upper(task_name)))):
            subp.run(['cp',os.path.join(path_root,pedig_file_ptrn.format(str.upper(task_name))),path_pleio_contrast]) 
        
        pedig_f = os.path.join(path_pleio_contrast,pedig_file_ptrn.format(str.upper(task_name)))
        
        # save pheno cov to file
        pheno_f= os.path.join(path_pleio_contrast,'pheno_cov.csv')
        if not os.path.isfile(pheno_f):
            sbt_pheno_df[['ID','Age_in_Yrs','Gender',pheno_1,pheno_2,cov_1]].to_csv(pheno_f,index=False)   
        
        # collect all pleio contrasts in dictionary
        pleio_asso.append(Pleio_Asso(out_dir = path_pleio_contrast,
                                      pedig_f = pedig_f,
                                      pheno_f = pheno_f,
                                      var_1 = pheno_1,
                                      var_2 = pheno_2,
                                      covar_1 = cov_1)) 

2bk Cluster_1 sub1
2bk Cluster_1 sub2
2bk Cluster_1 sub3
2bk Cluster_1 sub4
2bk Cluster_1 sub5
2bk Cluster_2 sub1
2bk Cluster_2 sub2
2bk Cluster_2 sub3
2bk Cluster_2 sub4
2bk Cluster_2 sub5
2bk Cluster_3 sub1
2bk Cluster_3 sub2
2bk Cluster_3 sub3
2bk Cluster_3 sub4
2bk Cluster_3 sub5
2bk Cluster_4 sub1
2bk Cluster_4 sub2
2bk Cluster_4 sub3
2bk Cluster_4 sub4
2bk Cluster_4 sub5
2bk Cluster_5 sub1
2bk Cluster_5 sub2
2bk Cluster_5 sub3
2bk Cluster_5 sub4
2bk Cluster_5 sub5
2bk Cluster_6 sub1
2bk Cluster_6 sub2
2bk Cluster_6 sub3
2bk Cluster_6 sub4
2bk Cluster_6 sub5
2bk Cluster_7 sub1
2bk Cluster_7 sub2
2bk Cluster_7 sub3
2bk Cluster_7 sub4
2bk Cluster_7 sub5
2bk Cluster_8 sub1
2bk Cluster_8 sub2
2bk Cluster_8 sub3
2bk Cluster_8 sub4
2bk Cluster_8 sub5
2bk Cluster_9 sub1
2bk Cluster_9 sub2
2bk Cluster_9 sub3
2bk Cluster_9 sub4
2bk Cluster_9 sub5
2bk Cluster_10 sub1
2bk Cluster_10 sub2
2bk Cluster_10 sub3
2bk Cluster_10 sub4
2bk Cluster_10 sub5
2bk Cluster_11 sub1
2bk Cluster_11 sub2
2bk C

In [88]:
def run_pleio(x):
    subp.run(['bash', os.path.join(x.out_dir,'pleio_run.sh'),
              x.out_dir,
              x.pedig_f,
              x.pheno_f,
              x.var_1,
              x.var_2,
              x.covar_1,
             ])
    
    # collect result
    RhoE = ''
    RhoE_pval=''
    RhoG = ''
    RhoG_pval_0 = ''
    RhoG_pval_1 = ''
    results = ''
    
    contrast_name = '{}_{}'.format(x.var_1,x.var_2)
    fp = open(os.path.join(x.out_dir,'solar_pleio.out'))
    for i,line in enumerate(fp):
        if 'CONVERGENCE FAILURE' in line:
            print('{}_{}'.format(x.var1,x.var_2))
            print(line)
            results = {'contrast_name' : contrast_name,'no_converg' : True}
            break
        if 'RhoE is ' in line:
            RhoE = float(line.strip('\n').split(' ')[3])
            RhoE_pval = float(line.strip('\n').split(' ')[-1])
        if 'RhoG is ' in line:
            RhoG = float(line.strip('\n').split(' ')[-1])
            #print(out_dir)
            #print(line.strip('\t ').strip('\n'))
        if 'RhoG different from zero' in line:
            RhoG_pval_0 = float(line.strip('\n').split(' ')[-1])
            #print(line.strip('\t '))
        if 'RhoG different from -1.0' in line:
            RhoG_pval_1 = float(line.strip('\n').split(' ')[-1])
            results = {'contrast_name' : contrast_name,'no_converg' : False,
                       'RhoE' : RhoE,
                       'RhoE_pval' : RhoE_pval,
                       'RhoG' : RhoG,
                       'RhoG_pval_0' : RhoG_pval_0,
                       'RhoG_pval_1' : RhoG_pval_1
                      }
            break
        elif 'RhoG different from 1.0' in line:
            RohG_pval_1 = float(line.strip('\n').split(' ')[-1])
            results = {'contrast_name' : contrast_name,'no_converg' : False,
                       'RhoE' : RhoE,
                       'RhoE_pval' : RhoE_pval,
                       'RhoG' : RhoG,
                       'RhoG_pval_0' : RhoG_pval_0,
                       'RhoG_pval_1' : RhoG_pval_1
                      }
            break
    return results

In [89]:
# function from http://danshiebler.com/2016-09-14-parallel-progress-bar/
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed

def parallel_process(array, function, n_jobs=12, use_kwargs=False, front_num=3):
    """
        A parallel version of the map function with a progress bar. 

        Args:
            array (array-like): An array to iterate over.
            function (function): A python function to apply to the elements of array
            n_jobs (int, default=16): The number of cores to use
            use_kwargs (boolean, default=False): Whether to consider the elements of array as dictionaries of 
                keyword arguments to function 
            front_num (int, default=3): The number of iterations to run serially before kicking off the parallel job. 
                Useful for catching bugs
        Returns:
            [function(array[0]), function(array[1]), ...]
    """
    #We run the first few iterations serially to catch bugs
    if front_num > 0:
        front = [function(**a) if use_kwargs else function(a) for a in array[:front_num]]
    #If we set n_jobs to 1, just run a list comprehension. This is useful for benchmarking and debugging.
    if n_jobs==1:
        return front + [function(**a) if use_kwargs else function(a) for a in tqdm(array[front_num:])]
    #Assemble the workers
    with ProcessPoolExecutor(max_workers=n_jobs) as pool:
        #Pass the elements of array into function
        if use_kwargs:
            futures = [pool.submit(function, **a) for a in array[front_num:]]
        else:
            futures = [pool.submit(function, a) for a in array[front_num:]]
        kwargs = {
            'total': len(futures),
            'unit': 'it',
            'unit_scale': True,
            'leave': True
        }
        #Print out the progress as tasks complete
        for f in tqdm(as_completed(futures), **kwargs):
            pass
    out = []
    #Get the results from the futures. 
    for i, future in tqdm(enumerate(futures)):
        try:
            out.append(future.result())
        except Exception as e:
            out.append(e)
    return front + out

In [90]:
# run one job for debuging
subp.run(['bash', os.path.join(pleio_asso[0].out_dir,'pleio_run.sh'),
              pleio_asso[0].out_dir,
              pleio_asso[0].pedig_f,
              pleio_asso[0].pheno_f,
              pleio_asso[0].var_1,
              pleio_asso[0].var_2
         ])

CompletedProcess(args=['bash', '/scratch/yassinebha/pleio/pleio_association/pleio_2018-08-06_wm/2bk_sub5/pleio_run.sh', '/scratch/yassinebha/pleio/pleio_association/pleio_2018-08-06_wm/2bk_sub5', '/scratch/yassinebha/pleio/pleio_association/pleio_2018-08-06_wm/2bk_sub5/solar_WM_spm_pedigre.csv', '/scratch/yassinebha/pleio/pleio_association/pleio_2018-08-06_wm/2bk_sub5/pheno_cov.csv', 'Cluster_1', 'WM_2bk_sub5'], returncode=0)

In [91]:
Results = parallel_process(pleio_asso,run_pleio,use_kwargs=False)

100%|██████████| 36.0/36.0 [00:00<00:00, 89.8it/s]
36it [00:00, 198677.56it/s]


In [ ]:
for ix_trial, trial in enumerate(list_trial):
    for ix_clust, clust in enumerate(list_pheno_clust):
        for ix_subt, subt in enumerate(list_sbt):

In [ ]:
#collect results
RohG= np.eye(len(pheno_clean_df.columns.drop('ID')))
RohG_pval_0 = np.eye(len(pheno_clean_df.columns.drop('ID')))
RohG_pval_1 = np.eye(len(pheno_clean_df.columns.drop('ID')))
RohE= np.eye(len(pheno_clean_df.columns.drop('ID')))
RohE_pval = np.eye(len(pheno_clean_df.columns.drop('ID')))
RohP= np.eye(len(pheno_clean_df.columns.drop('ID')))
RohP_pval = np.eye(len(pheno_clean_df.columns.drop('ID')))
count = 0
No_converge = collections.namedtuple('No_converge', [
    'var_1',
    'var_2',
    'pedig_f',
    'pheno_f',
    'out_dir'
])
no_converges = []
pedig_f = os.path.join(path_root,'pleio_all_pheno_pedig.csv')
for ix_pheno_1, pheno_1 in enumerate(pheno_clean_df.columns.drop('ID')):
    for ix_pheno_2, pheno_2 in enumerate(pheno_clean_df.columns.drop('ID')):
        if pheno_1  == pheno_2:
            continue
        out_dir = '{}_{}'.format(pheno_1,pheno_2)
        path_pleio_contrast = os.path.join(path_pleio,'{}_{}'.format(pheno_1,pheno_2))
        pedig_f = os.path.join(path_pleio_contrast,'pleio_all_pheno_pedig.csv')
        pheno_f= os.path.join(path_pleio_contrast,'pheno_cov.csv')
        
        #number_lines = sum(1 for line in open(os.path.join(path_root,out_dir,'solar_pleio.out')))
        fp = open(os.path.join(path_pleio,out_dir,'solar_pleio.out'))
        for i,line in enumerate(fp):
            if 'CONVERGENCE FAILURE' in line:
                print(out_dir)
                print(line)
                count+=1
                no_converges.append(No_converge(var_1 = pheno_1,
                                                var_2 = pheno_2,
                                                out_dir = out_dir,
                                                pedig_f = pedig_f,
                                                pheno_f = pedig_f))
                break
                
            if 'RhoE is ' in line:
                RohE[ix_pheno_1,ix_pheno_2] = float(line.strip('\n').split(' ')[3])
                RohE_pval[ix_pheno_1,ix_pheno_2] = float(line.strip('\n').split(' ')[-1])
                
            if 'RhoG is ' in line:
                RohG[ix_pheno_1,ix_pheno_2] = float(line.strip('\n').split(' ')[-1])
                
                #print(out_dir)
                #print(line.strip('\t ').strip('\n'))
            if 'RhoG different from zero' in line:
                RohG_pval_0[ix_pheno_1,ix_pheno_2] = float(line.strip('\n').split(' ')[-1])
                
                #print(line.strip('\t '))
            if 'RhoG different from -1.0' in line:
                RohG_pval_1[ix_pheno_1,ix_pheno_2] = float(line.strip('\n').split(' ')[-1])
                
            elif 'RhoG different from 1.0' in line:
                RohG_pval_1[ix_pheno_1,ix_pheno_2] = float(line.strip('\n').split(' ')[-1])
                
            if 'RhoP is ' in line:
                #print(float(line.strip('\n').split(' ')[-1]))
                RohP[ix_pheno_1,ix_pheno_2] = float(line.strip('\n').split(' ')[-1])
                
            if 'RhoP different from zero' in line:
                RohP_pval[ix_pheno_1,ix_pheno_2] = float(line.strip('\n').split(' ')[-1])
                #print(float(line.strip('\n').split(' ')[-1]))
                break           

In [ ]:
# import and merge all weight and pheno data
dict_all_weight_pheno = {}
dict_all_weight_pheno_clust = {}
dict_all_weight_pheno_clust_json = {}

for ind_f , task_folder in enumerate(list_subtype_folder):
    
    # set path and task name
    path_subtype = os.path.join(path_root,task_folder);
    path_association =  os.path.join(path_subtype,'associations/');
    path_networks =  os.path.join(path_subtype,'networks/');
    # number of subtype
    nb_sbt = natural_keys(path_subtype)[9]
    # task name
    task_name = natural_keys(path_subtype)[10][5:-1]
    # List phenotypes
    list_pheno  = [f for f in os.listdir(path_association)]
    list_pheno.sort(key=natural_keys)
    # List trials
    list_trial  = [f for f in os.listdir(path_networks)]
    # List subtype
    list_subtype = ['sub{}'.format(ii) for ii in range(1,nb_sbt+1)]
    
    # collect weight data
    for ind_t, trial_name in enumerate(list_trial) :
        # collect all pheno data
        all_pheno_clust = []
        names_pheno = []
        for ix, pheno_name in enumerate(list_pheno):
            #from IPython.core.debugger import Tracer; Tracer()() 
            mat_file = os.path.join(path_association,pheno_name,'association_stats_{}.mat'.format(pheno_name))
            %octave_push mat_file
            %octave mat_load = load(mat_file);
            %octave_pull mat_load
            model_x_norm = mat_load['model_norm']['x']
            model_labels_y_norm = mat_load['model_norm']['labels_y']
            model_labels_x_norm = mat_load['model_norm']['labels_x']

            my_pheno = np.array(model_x_norm[:,1])
            my_pheno_name= model_labels_y_norm[1]
            all_pheno_clust.append(my_pheno)
            names_pheno.append(my_pheno_name)

        # create pheno dataframe
        all_pheno_clust = np.concatenate([model_labels_x_norm[...,None],np.transpose(all_pheno_clust)],axis=1)
        all_pheno_clust_df = pd.DataFrame(all_pheno_clust,columns=np.append('ID',  list_pheno))
        all_pheno_clust_df['ID'] = all_pheno_clust_df.ID.str.strip()

        #collect weight and create dataframe
        weight_file = os.path.join(path_networks,'{}/sbt_weights_net_{}.csv'.format(trial_name,trial_name))
        weight_df = pd.read_csv(weight_file)
        column_names =np.append('ID',  list_subtype)
        weight_df.columns= column_names
        weight_df['ID'] = weight_df.ID.str.strip()

        # merge weight amd pheno dataframe
        weight_pheno_clust_df=pd.merge(weight_df,all_pheno_clust_df,on='ID',how='left')
        # save it csv
        weight_pheno_clust_df.to_csv(os.path.join(path_subtype,'{}_weight_pheno_subtype.csv'.format(trial_name)))

        # stack to dictionary
        dict_all_weight_pheno_clust[task_name + "_" + 
                                    trial_name + "_" + 
                                    str(nb_sbt) + '_subtypes'] = weight_pheno_clust_df
        # stack to json to be saved later
        dict_all_weight_pheno_clust_json[task_name + "_" +
                                         trial_name + "_" +
                                         str(nb_sbt) + '_subtypes'] = weight_pheno_clust_df.to_json(orient='split')
        # Merge all pheno with subtype weights
        all_pheno_pruned = pd.merge(weight_pheno_clust_df[['ID']+list_subtype],all_pheno,on='ID',how='left')
        # Drop NaN
        all_pheno_pruned.dropna(inplace=True)
        dict_all_weight_pheno[task_name + "_" +
                              trial_name + "_" +
                              str(nb_sbt) + '_subtypes'] = all_pheno_pruned